# 聚合

基本语法:

```
db.colllection.aggregate([
    {'$match': {expressions}},
    {'$group': {'new_field1': 'field1', 'new_field2': 'field2', ....}},
    {'$project': {}},
    {'$limit': 5},
    {'$sort': {'new_field1': 1, 'new_field2': -1}
    ])
```

- `$match` 阶段为第一阶段匹配，相当于sql中的`where子句`
- `$group` 阶段为分组，相当于sql中的`group by子句`

In [2]:
from pymongo import MongoClient
from datetime import datetime
from bson import ObjectId

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
db = client['purchase']

In [5]:
db.list_collection_names()

['student', 'scores', 'subsort', 'sort', 'product']

In [6]:
product = db.product
sort = db.sort
subsort = db.subsort

In [8]:
agg = product.aggregate([
    {'$match': {'Price': {'$gt': 100}, 'Product_Place': {'$type': 'string'}}},
    {'$group': {'_id': '$Product_Place', 'avg_price': {'$avg': '$Price'}, 'count': {'$sum':1}}},
    {'$sort': {'count': -1}},
    {'$project': {'count': 1}},
    {'$limit': 10}
])
list(agg)

[{'_id': '北京', 'count': 226},
 {'_id': '河北', 'count': 100},
 {'_id': '宁波', 'count': 84},
 {'_id': '广东', 'count': 81},
 {'_id': '国产', 'count': 71},
 {'_id': '上海', 'count': 55},
 {'_id': '深圳', 'count': 47},
 {'_id': '广州', 'count': 41},
 {'_id': '日本', 'count': 41},
 {'_id': '马来西亚', 'count': 34}]

In [9]:
product.distinct('Product_Place', {'Price': {'$gt': 100}})

['天津',
 '江苏常熟',
 '日本',
 None,
 '国产',
 '日本/深圳',
 '马来西亚',
 '北京',
 '广州',
 '浙江',
 '上海',
 '深圳',
 '大连',
 '广东',
 '泰国',
 '山东',
 '杭州',
 '敖江',
 '河北',
 '香港',
 '宁波',
 '顺德',
 '中国',
 '法国',
 '常熟',
 '波多黎哥/爱尔兰/新加坡',
 '不定',
 '美国',
 '马来西亚/日本/美国/墨西哥',
 '菲律宾',
 '韩国',
 '汕头',
 '珠海',
 '惠州',
 '济南',
 '巴西',
 '台湾',
 '德国',
 '南韩',
 '安徽',
 '厦门',
 '英国',
 '福建',
 '江苏']

### `$lookup`实现左连接查询

```python
db.collection.aggregate{
   '$lookup':
     {
       from: <collection to join>,
       localField: <field from the input documents>,
       foreignField: <field from the documents of the "from" collection>,
       as: <output array field>
     }
}
```


In [10]:
subsort.find_one()

{'_id': ObjectId('5bc8a96e8c93482510fe2dab'),
 'SubSort_ID': '1101',
 'SubSort_name': '复印机',
 'Sort_ID': '11'}

In [11]:
sort.find_one()

{'_id': ObjectId('5bc8a95a8c93482510fe2d8d'),
 'Sort_ID': '11',
 'Sort_name': '办公机器设备'}

In [12]:
sort_subsort = subsort.aggregate([
    {'$lookup':{
    'from': "sort",
    'localField': 'Sort_ID',
    'foreignField': 'Sort_ID',
    'as': 'sort_info'}},
    {'$unwind': '$sort_info'},
    {'$project': {
         'SubSort_ID': True,
         'SubSort_name': True,
         'Sort_ID':True,
         'sort_info.Sort_ID': True,
         'sort_info.Sort_name': True,
         '_id': False
     }},
     {'$limit': 10}])

In [13]:
for s in sort_subsort:
    print(s)

{'SubSort_ID': '1101', 'SubSort_name': '复印机', 'Sort_ID': '11', 'sort_info': {'Sort_ID': '11', 'Sort_name': '办公机器设备'}}
{'SubSort_ID': '1102', 'SubSort_name': '一体机', 'Sort_ID': '11', 'sort_info': {'Sort_ID': '11', 'Sort_name': '办公机器设备'}}
{'SubSort_ID': '1103', 'SubSort_name': '复合机', 'Sort_ID': '11', 'sort_info': {'Sort_ID': '11', 'Sort_name': '办公机器设备'}}
{'SubSort_ID': '1104', 'SubSort_name': '塑封机', 'Sort_ID': '11', 'sort_info': {'Sort_ID': '11', 'Sort_name': '办公机器设备'}}
{'SubSort_ID': '1105', 'SubSort_name': '装裱机', 'Sort_ID': '11', 'sort_info': {'Sort_ID': '11', 'Sort_name': '办公机器设备'}}
{'SubSort_ID': '1106', 'SubSort_name': '装订机', 'Sort_ID': '11', 'sort_info': {'Sort_ID': '11', 'Sort_name': '办公机器设备'}}
{'SubSort_ID': '1107', 'SubSort_name': '碎纸机', 'Sort_ID': '11', 'sort_info': {'Sort_ID': '11', 'Sort_name': '办公机器设备'}}
{'SubSort_ID': '1108', 'SubSort_name': '交换机', 'Sort_ID': '11', 'sort_info': {'Sort_ID': '11', 'Sort_name': '办公机器设备'}}
{'SubSort_ID': '1109', 'SubSort_name': '电话机', 'Sort_ID':